In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import regex as re
from lxml import etree as et
from itertools import repeat
import csv
import random
import time
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

In [ ]:
header_list = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML,                           like Gecko) Chrome/103.0.5060.66 Safari/537.36",
              "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:53.0) Gecko/20100101 Firefox/53.0",
              "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393"]

base_url = "https://www.amazon.in/s?k="
laptop_brands = ['Apple', 'HP', 'Lenovo', 'Acer', 'Dell', 'MSI', 'Asus', 'Microsoft', 'Razer', 'Samsung', 'Alienware', 'Honor', 'Chromebook', 'Ultrabook', 'Notebook', 'Xiaomi', 'RGY', 'AVITA', 'Fujitsu']
product_list = []
d = 0

In [ ]:
for brand in tqdm_notebook(laptop_brands):
  for page_no in range(4):
    page_url = base_url + brand + '-laptop-' '&page=' + str(page_no)
    user_agent = random.choice(header_list)
    header = {"User-Agent": user_agent}
    webpage = requests.get(page_url, headers = header)
    if webpage.status_code == 200:
      soup1 = BeautifulSoup(webpage.content, 'html.parser')
      soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
      divs = soup2.find_all('div', {'class' : 'sg-col-20-of-24 s-matching-dir sg-col-16-of-20 sg-col sg-col-8-of-12 sg-col-12-of-16'} )
      for div in divs:
        swlink = div.find_all('a', {'class' : 'a-link-normal s-no-outline'})
        for l in swlink:
          product_list.append('https://www.amazon.in' + l.get('href'))
    else:
      d += 1

  0%|          | 0/19 [00:00<?, ?it/s]

In [ ]:
print(len(product_list), d)

256 60


In [ ]:
cols = ['Url', 'Brand', 'Current Price', 'Original Price', 'Discount Percentage', 
        'Rating', 'Number of Ratings', 'Picture', 'Highlights',
        'Reviews']

feature_list = ['Model', 'Model Name', 'Model Year', 'Product Dimensions', 'Ram Memory Installed Size', 
                'Ram Memory Technology', 'Hard Drive Size', 'Hard Drive Interface','Operating System', 
                'Processor Brand', 'Processor Speed', 'Processor Type', 'Graphics Card Description',
                'Graphics RAM Type', 'Standing screen display size', 'Display Type', 'Resolution', 
                'Battery Average Life', 'Battery cell composition', 'Refresh Rate', 'Device type', 
                'Country of Origin', 'Item Weight', 'ASIN']

cols.extend(feature_list)
kagg_lap = pd.DataFrame(columns = cols)

In [ ]:
def feature_result_transform(feature_list, feature_result_dict):
  
  result_list = []
  result_list.extend(repeat(np.nan, len(feature_list)))

  for key in feature_result_dict.keys():
    for i in range(len(feature_list)):
      if feature_list[i] == key:
        result_list[i] = feature_result_dict[key]   
    
  return result_list

In [ ]:
for link in tqdm_notebook(product_list):
  url = link
  user_agent = random.choice(header_list)
  header = {"User-Agent": user_agent}
  webpage = requests.get(url, headers = header)
  if webpage.status_code == 200:
    soup1 = BeautifulSoup(webpage.content, 'html.parser')
    soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

    try:
        brand = soup2.find('tr', {'class' : 'a-spacing-small po-brand'})
        brand = brand.get_text().strip().split()[-1]
    except:
        brand = np.nan
        pass

    prices = soup2.find(id = 'apex_desktop')
    try:
      prices = prices.find_all('span', {'class' : 'a-offscreen'})
      cur_price = int(re.sub('₹|,', '', prices[0].get_text().strip()))
      org_price = int(re.sub('₹|,', '', prices[1].get_text().strip()))
      discount = ((org_price - cur_price)/org_price) * 100  
    except:
      cur_price = np.nan
      org_price = np.nan
      discount = np.nan
    #print(cur_price, org_price, link)

    # try:
    #   nmodels = soup2.find(id = 'variation_color_name')
    #   nmodels = len(nmodels.find_all('li'))
    # except:
    #   nmodels = np.nan
    #print(nmodels)

    try:
      pic = soup2.find(id = 'landingImage').get('src')
    except:
      pic = np.nan
    #print(pic)

    try:
      highlights = soup2.find('ul', {'class' : 'a-unordered-list a-vertical a-spacing-mini'})
      highlights = highlights.find_all('span', {'class' : 'a-list-item'})
      highlights = [h.get_text().strip() for h in highlights]
    except:
      highlights = np.nan
    #print(highlights)

    try:
      table = soup2.find(id = 'prodDetails')

      features = table.find_all('th', {'class': "a-color-secondary a-size-base prodDetSectionEntry"})
      features = [f.get_text().strip() for f in features]
      
      results = table.find_all('td', {'class' : 'a-size-base prodDetAttrValue'})
      results = [r.get_text().strip().replace('\u200e', '') for r in results]
      
      dicted = {}
      for i in range(len(features)):
        dicted[features[i]] = results[i]    
      #print(dicted)
      #results = feature_result_transform(dicted)
    except:
      #print('---')
      dicted  = {}
      pass

    try:
      revs = soup2.find('div', {'class' : 'card-padding'})
      revs = revs.find_all('div', {'class' : 'a-row a-spacing-small review-data'})
      reviews = [rev.get_text().strip().replace('\n', '') for rev in revs]
    except:
      reviews = np.nan
    
    try:
      cust_revs_box = soup2.find('span', {'class' : 'cr-widget-TitleRatingsAndHistogram'})
      text_rating = cust_revs_box.find('span', {'class' : 'a-size-medium a-color-base'}).get_text().strip()
      nglobal_ratings = cust_revs_box.find('span', {'class' : 'a-size-base a-color-secondary'}).get_text().strip()
      nglobal_ratings = int(re.sub(',', '', nglobal_ratings.split()[0]))
    except:
      text_rating = np.nan
      nglobal_ratings = np.nan

    results = feature_result_transform(feature_list, dicted)
    row = [url, brand, cur_price, org_price, discount, text_rating, nglobal_ratings, pic, highlights, reviews]
    row.extend(results)
    kagg_lap.loc[len(kagg_lap.index)] = row   
    #print(results)


  0%|          | 0/256 [00:00<?, ?it/s]

In [ ]:
print(len(row), len(results), len(kagg_lap.columns))

34 24 34


No model, model name

In [ ]:
kagg_lap.to_csv('kagg_lap3.csv')

In [ ]:
kagg_lap.head()

,Url,Brand,Current Price,Original Price,Discount Percentage,Rating,Number of Ratings,Picture,Highlights,Reviews,...,Standing screen display size,Display Type,Resolution,Battery Average Life,Battery cell composition,Refresh Rate,Device type,Country of Origin,Item Weight,ASIN
0,https://www.amazon.in/HP-Laptop-Ryzen-Quad-Cor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.amazon.in/HP-ProBook-440-G8-i5-113...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.amazon.in/Renewed-Laptop-Elitebook...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.amazon.in/HP-5-5600H-Graphics-Resp...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.amazon.in/HP-Pavilion-Micro-Edge-M...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
url = product_list[-100]
user_agent = random.choice(header_list)
header = {"User-Agent": user_agent}
webpage = requests.get(url, headers = header)
print(webpage.status_code)
if webpage.status_code == 200:
    soup1 = BeautifulSoup(webpage.content, 'html.parser')
    soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')
    print(webpage.status_code)

503


In [ ]:
soup2

In [ ]:
print(url)

https://www.amazon.in/ASUS-17-3-inch-RTX-3050-Ti-Graphics-G713IE-HX040W/dp/B0B3DQ36P7/ref=sr_1_13?keywords=Ultrabook-laptop-&qid=1679919022&sr=8-13


In [ ]:
prices = soup2.find(id = 'apex_desktop')
prices = prices.find_all('span', {'class' : 'a-offscreen'})
cur_price = int(re.sub('₹|,', '', prices[0].get_text().strip()))
org_price = int(re.sub('₹|,', '', prices[1].get_text().strip()))
discount = ((org_price - cur_price)/org_price) * 100
print(cur_price, org_price, discount)

AttributeError: ignored

In [ ]:
nmodels = soup2.find(id = 'variation_color_name')
nmodels = len(nmodels.find_all('li'))
print(nmodels)

In [ ]:
pic = soup2.find(id = 'landingImage').get('src')
print(pic)

In [ ]:
highlights = soup2.find('ul', {'class' : 'a-unordered-list a-vertical a-spacing-mini'})
highlights = highlights.find_all('span', {'class' : 'a-list-item'})
highlights = [h.get_text().strip() for h in highlights]
print(highlights)

In [ ]:
revs = soup2.find('div', {'class' : 'card-padding'})
revs = revs.find_all('div', {'class' : 'a-row a-spacing-small review-data'})
reviews = [rev.get_text().strip().replace('\n', '') for rev in revs]
print(reviews)

In [ ]:
cust_revs_box = soup2.find('span', {'class' : 'cr-widget-TitleRatingsAndHistogram'})
text_rating = cust_revs_box.find('span', {'class' : 'a-size-medium a-color-base'}).get_text().strip()
nglobal_ratings = cust_revs_box.find('span', {'class' : 'a-size-base a-color-secondary'}).get_text().strip()
nglobal_ratings = int(re.sub(',', '', nglobal_ratings.split()[0]))
print(text_rating, nglobal_ratings)

In [ ]:
table = soup2.find(id = 'prodDetails')
print(table)
features = table.find_all('th', {'class': "a-color-secondary a-size-base prodDetSectionEntry"})
features = [f.get_text().strip() for f in features]

In [ ]:
a = soup2.find('tr', {'class' : 'a-spacing-small po-brand'})
a.get_text().strip().split()[-1]

In [ ]:
features

In [ ]:
table

In [ ]:
soup2.find(id = 'productDetails_feature_div')